In [13]:
import pandas as pd
import sqlite3
from google.colab import files



In [2]:
excel_file = "DEI_survey.xlsx"

QA_df = pd.read_excel(excel_file, sheet_name = 'DEI Questions')

print(QA_df.head())


  Question_Id                                          Questions  \
0    Aug_D_Q1                      Our company values diversity.   
1    Aug_D_Q2               Our company has a diverse workforce.   
2    Aug_D_Q3  Our company is taking actions to create a dive...   
3    Aug_D_Q4  I believe I can advance in my career regardles...   
4    Aug_D_Q5  Our company does not tolerate any incidents of...   

   Strongly Disagree  Disagree  Neutral  Agree  Strongly Agree  
0                NaN       NaN      NaN    NaN             NaN  
1                NaN       NaN      NaN    NaN             NaN  
2                NaN       NaN      NaN    NaN             NaN  
3                NaN       NaN      NaN    NaN             NaN  
4                NaN       NaN      NaN    NaN             NaN  


In [3]:
data_df = pd.read_excel(excel_file, sheet_name = 'DEI')

print(data_df.head())

   Id      Name   Surname   Division Manager       Gender Sexual_Orientation  \
0   1   Deborah   Addison         IT      No  Transgender       Heterosexual   
1   2      Mona      Hill  Marketing      No         Male       Heterosexual   
2   3  Kimberly   Shelton    Finance      No       Female       Heterosexual   
3   4     Denis  Robinson         HR      No         Male       Heterosexual   
4   5    Carmen      Gunn         RD      No         Male       Heterosexual   

               LGBTQ Indigenous Ethnicity  ... E_Neutral E_Positive Aug_I_Q1  \
0  Prefer not to say         No     White  ...         1          2        0   
1                 No        Yes     White  ...         2          3        0   
2                 No         No     White  ...         0          4       -2   
3                 No         No     White  ...         1          3        0   
4                 No         No     Asian  ...         0          5        1   

  Aug_I_Q2  Aug_I_Q3 Aug_I_Q4 Aug_I_Q5

In [4]:
employees_cols = [
    'Id', 'Name', 'Surname', 'Division', 'Manager', 'Gender', 'Sexual_Orientation',
    'LGBTQ', 'Indigenous', 'Ethnicity', 'Disability', 'Minority', 'Veteran',
    'Date of Birth', 'Age', 'Preferred Name', 'Nationality', 'Hobbies', 'Pronouns',
    'Mobile Number', 'Email'
]

employees_df = data_df[employees_cols].copy()

# Rename columns for SQL
employees_df.columns = [
    'employee_id', 'name', 'surname', 'division', 'manager', 'gender', 'sexual_orientation',
    'LGBTQ', 'indigenous', 'ethnicity', 'disability', 'minority', 'veteran', 'date_of_birth',
    'age', 'preferred_name', 'nationality', 'hobbies', 'pronouns', 'mobile_number', 'email'
]

print(employees_df.head())

   employee_id      name   surname   division manager       gender  \
0            1   Deborah   Addison         IT      No  Transgender   
1            2      Mona      Hill  Marketing      No         Male   
2            3  Kimberly   Shelton    Finance      No       Female   
3            4     Denis  Robinson         HR      No         Male   
4            5    Carmen      Gunn         RD      No         Male   

  sexual_orientation              LGBTQ indigenous ethnicity  ... minority  \
0       Heterosexual  Prefer not to say         No     White  ...       No   
1       Heterosexual                 No        Yes     White  ...      Yes   
2       Heterosexual                 No         No     White  ...       No   
3       Heterosexual                 No         No     White  ...      Yes   
4       Heterosexual                 No         No     Asian  ...       No   

  veteran date_of_birth age  preferred_name nationality     hobbies  \
0      No    1993-06-04  29         Deb

In [5]:
# Keeping only the columns we need
questions_df = QA_df[['Question_Id', 'Questions']].copy()

# Rename columns
questions_df = questions_df.rename(columns={
    'Question_Id': 'question_id',
    'Questions': 'question'
})

# Add dimension based on question_id prefix
def get_dimension(qid):
    if qid.startswith('Aug_D'):
        return 'Diversity'
    elif qid.startswith('Aug_E'):
        return 'Engagement'
    elif qid.startswith('Aug_I'):
        return 'Inclusion'
    else:
        return 'Unknown'

questions_df['dimension'] = questions_df['question_id'].apply(get_dimension)

# reorder columns
questions_df = questions_df[['question_id', 'dimension', 'question']]

print(questions_df.head())


  question_id  dimension                                           question
0    Aug_D_Q1  Diversity                      Our company values diversity.
1    Aug_D_Q2  Diversity               Our company has a diverse workforce.
2    Aug_D_Q3  Diversity  Our company is taking actions to create a dive...
3    Aug_D_Q4  Diversity  I believe I can advance in my career regardles...
4    Aug_D_Q5  Diversity  Our company does not tolerate any incidents of...


In [6]:
# Define question columns per dimension
dimensions = {
    'Diversity': ['Aug_D_Q1','Aug_D_Q2','Aug_D_Q3','Aug_D_Q4','Aug_D_Q5'],
    'Engagement': ['Aug_E_Q1','Aug_E_Q2','Aug_E_Q3','Aug_E_Q4','Aug_E_Q5'],
    'Inclusion': ['Aug_I_Q1','Aug_I_Q2','Aug_I_Q3','Aug_I_Q4','Aug_I_Q5']
}

detailed_survey = data_df[['Id']].copy()
detailed_survey = detailed_survey.rename(columns={'Id':'employee_id'})

all_question_cols = [col for cols in dimensions.values() for col in cols]
detailed_survey = pd.concat([detailed_survey, data_df[all_question_cols]], axis=1)

# Calculate totals per dimension and sentiment
for dim, cols in dimensions.items():
    detailed_survey[f'{dim}_total'] = data_df[cols].sum(axis=1)
    detailed_survey[f'{dim}_sentiment'] = detailed_survey[f'{dim}_total'].apply(
        lambda x: 'Negative' if x < 0 else ('Neutral' if x == 0 else 'Positive')
    )

ordered_cols = ['employee_id']
for dim, cols in dimensions.items():
    ordered_cols += cols
    ordered_cols.append(f'{dim}_total')
    ordered_cols.append(f'{dim}_sentiment')

detailed_survey = detailed_survey[ordered_cols]

print(detailed_survey.head())


   employee_id  Aug_D_Q1  Aug_D_Q2  Aug_D_Q3  Aug_D_Q4  Aug_D_Q5  \
0            1        -1         2        -1         0        -2   
1            2        -2         2         0         2         2   
2            3         0        -1        -2        -1         1   
3            4        -2         0         1         0        -2   
4            5        -2        -1        -1        -1         2   

   Diversity_total Diversity_sentiment  Aug_E_Q1  Aug_E_Q2  ...  Aug_E_Q5  \
0               -2            Negative         0        -1  ...         1   
1                4            Positive         2         2  ...         0   
2               -3            Negative         2         2  ...         2   
3               -3            Negative         1         2  ...         1   
4               -3            Negative         1         2  ...         1   

   Engagement_total  Engagement_sentiment  Aug_I_Q1 Aug_I_Q2  Aug_I_Q3  \
0                -1              Negative         0   

## SQL Tables Created

We have created **three tables** for the DEI survey project:

### 1. `employees`
Stores employee information.

| Column Name         | Description                  |
|--------------------|------------------------------|
| employee_id        | Unique ID for each employee  |
| name               | Employee's first name        |
| surname            | Employee's last name         |
| division           | Employee's division          |
| manager            | Manager name                 |
| gender             | Gender of the employee       |
| sexual_orientation | Sexual orientation           |
| LGBTQ              | LGBTQ status                 |
| indigenous         | Indigenous status            |
| ethnicity          | Ethnicity                    |
| disability         | Disability status            |
| minority           | Minority status              |
| veteran            | Veteran status               |
| date_of_birth      | Date of birth                |
| age                | Age of employee              |
| preferred_name     | Preferred name               |
| nationality        | Nationality                  |
| hobbies            | Hobbies                      |
| pronouns           | Pronouns                     |
| mobile_number      | Contact number               |
| email              | Email address                |

---

### 2. `questions`
Stores survey questions and their dimension.

| Column Name | Description                     |
|------------|---------------------------------|
| question_id | Unique ID for each question    |
| dimension   | Dimension (Diversity/Engagement/Inclusion) |
| question    | Text of the question           |

---

### 3. `detailed_survey`
Stores individual question responses, totals, and sentiment.

| Column Name       | Description                       |
|------------------|-----------------------------------|
| employee_id       | Unique ID for each employee       |
| Aug_D_Q1 ... Q5   | Responses to Diversity questions  |
| Diversity_total   | Total score for Diversity          |
| Diversity_sentiment | Sentiment for Diversity (Positive/Neutral/Negative) |
| Aug_E_Q1 ... Q5   | Responses to Engagement questions |
| Engagement_total  | Total score for Engagement        |
| Engagement_sentiment | Sentiment for Engagement       |
| Aug_I_Q1 ... Q5   | Responses to Inclusion questions  |
| Inclusion_total   | Total score for Inclusion         |
| Inclusion_sentiment | Sentiment for Inclusion         |

---

We will now proceed with **data preprocessing and cleaning** to ensure the data is consistent, formatted correctly, and ready for analysis.


In [7]:
# ============================
# Cleaning employees_df
# ============================

# Strip spaces and fix text formatting
employees_df['name'] = employees_df['name'].str.strip().str.title()
employees_df['surname'] = employees_df['surname'].str.strip().str.title()
employees_df['division'] = employees_df['division'].str.strip().str.title()
employees_df['manager'] = employees_df['manager'].str.strip().str.title()
employees_df['email'] = employees_df['email'].str.strip().str.lower()
employees_df['pronouns'] = employees_df['pronouns'].str.strip()

# Convert DOB to datetime and recalculate age if needed
employees_df['date_of_birth'] = pd.to_datetime(employees_df['date_of_birth'], errors='coerce')
employees_df['age'] = ((pd.Timestamp.today() - employees_df['date_of_birth']).dt.days // 365)

# Drop duplicate or missing employee_id rows
employees_df = employees_df.drop_duplicates(subset=['employee_id'])
employees_df = employees_df[employees_df['employee_id'].notna()]



In [8]:
employees_df.head()


,employee_id,name,surname,division,manager,gender,sexual_orientation,LGBTQ,indigenous,ethnicity,...,minority,veteran,date_of_birth,age,preferred_name,nationality,hobbies,pronouns,mobile_number,email
0,1,Deborah,Addison,It,No,Transgender,Heterosexual,Prefer not to say,No,White,...,No,No,1993-06-04,32,Deborah,Polish,Travelling,They/them/theirs,363 436 1096,deborah.addison@mail.ca
1,2,Mona,Hill,Marketing,No,Male,Heterosexual,No,Yes,White,...,Yes,No,1963-08-02,62,Mona,Lao,Reading,He/him/his,905 980 9989,mona.hill@mail.ca
2,3,Kimberly,Shelton,Finance,No,Female,Heterosexual,No,No,White,...,No,No,1979-04-25,46,Kimberly,Canadian,Sports,She/her/hers,788 356 1577,kimberly.shelton@mail.ca
3,4,Denis,Robinson,Hr,No,Male,Heterosexual,No,No,White,...,Yes,No,1978-02-11,47,Denis,Canadian,Sports,He/him/his,946 855 5822,denis.robinson@mail.ca
4,5,Carmen,Gunn,Rd,No,Male,Heterosexual,No,No,Asian,...,No,No,1972-03-11,53,Carmen,Korean,Cooking,He/him/his,537 895 4532,carmen.gunn@mail.ca


In [9]:
employees_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   employee_id         10000 non-null  int64         
 1   name                10000 non-null  object        
 2   surname             10000 non-null  object        
 3   division            10000 non-null  object        
 4   manager             10000 non-null  object        
 5   gender              10000 non-null  object        
 6   sexual_orientation  10000 non-null  object        
 7   LGBTQ               10000 non-null  object        
 8   indigenous          10000 non-null  object        
 9   ethnicity           10000 non-null  object        
 10  disability          10000 non-null  object        
 11  minority            10000 non-null  object        
 12  veteran             10000 non-null  object        
 13  date_of_birth       10000 non-null  datetime64[

In [10]:
# Preprocess questions table
# ============================

print(questions_df.isnull().sum())

# Drop empty questions
questions_df = questions_df.dropna(subset=['question'])

question_id    0
dimension      0
question       0
dtype: int64


In [11]:
# ============================
# Preprocess detailed_survey table
# ============================

print(detailed_survey.isnull().sum())

# Ensure all question columns are numeric and fill missing with 0
detailed_survey[all_question_cols] = detailed_survey[all_question_cols].apply(pd.to_numeric, errors='coerce').fillna(0)

print(f"Number of duplicate employee IDs in detailed_survey: {detailed_survey.duplicated(subset=['employee_id']).sum()}")

# Drop duplicates if any
detailed_survey = detailed_survey.drop_duplicates(subset=['employee_id'])

employee_id             0
Aug_D_Q1                0
Aug_D_Q2                0
Aug_D_Q3                0
Aug_D_Q4                0
Aug_D_Q5                0
Diversity_total         0
Diversity_sentiment     0
Aug_E_Q1                0
Aug_E_Q2                0
Aug_E_Q3                0
Aug_E_Q4                0
Aug_E_Q5                0
Engagement_total        0
Engagement_sentiment    0
Aug_I_Q1                0
Aug_I_Q2                0
Aug_I_Q3                0
Aug_I_Q4                0
Aug_I_Q5                0
Inclusion_total         0
Inclusion_sentiment     0
dtype: int64
Number of duplicate employee IDs in detailed_survey: 0


In [12]:
# Save tables to SQLite
# ============================

conn = sqlite3.connect('dei_proj.db')

employees_df.to_sql('employees', conn, if_exists='replace', index=False)
questions_df.to_sql('questions', conn, if_exists='replace', index=False)
detailed_survey.to_sql('detailed_survey', conn, if_exists='replace', index=False)

conn.close()

print("Data preprocessing and cleaning completed. All three tables saved to SQLite.")

Data preprocessing and cleaning completed. All three tables saved to SQLite.


In [14]:
# Download the SQLite database file
files.download('dei_proj.db')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>